In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import KFold
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from joblib import dump

In [3]:
df_train = pd.read_csv("data/sign_mnist_train.csv")
df_test = pd.read_csv("data/sign_mnist_test.csv")
df = pd.concat([df_train, df_test], ignore_index=True)
print(df.shape)

(34627, 785)


In [ ]:
df.to_csv("data/all_train.csv", index=False)

In [ ]:
df_test_ASCII = pd.read_csv("../data/01_raw/test.csv")
df_test_ASCII_a = df_test_ASCII.iloc[:,1:785]
df_test_ASCII_b = df_test_ASCII.iloc[:,785:]

In [ ]:
columns = df_test.iloc[:,1:].columns
df_test_ASCII_a.columns = columns
df_test_ASCII_b.columns = columns

df_test_ASCII_a["id"] = df_test_ASCII["id"]
df_test_ASCII_b["id"] = df_test_ASCII["id"]

df_test_ASCII_ab = pd.concat([df_test_ASCII_a, df_test_ASCII_b], ignore_index=True)
print(df_test_ASCII_ab.shape)

In [ ]:
df_original = pd.DataFrame()
df_shuffled = pd.DataFrame()

df_original['hash'] = df.iloc[:,1:].apply(lambda row: hash(tuple(row)), axis=1)
df_original['label'] = df['label']
df_shuffled['hash'] = df_test_ASCII_ab.iloc[:,:-1].apply(lambda row: hash(tuple(row)), axis=1)
df_shuffled['id'] = df_test_ASCII_ab['id']

In [ ]:
merged_df = pd.merge(df_shuffled, df_original, on='hash')

In [ ]:
grouped_df = merged_df.groupby('id')['label'].agg(list).reset_index()
grouped_df[['label_a', 'label_b']] = pd.DataFrame(grouped_df['label'].tolist(), index=grouped_df.index)
grouped_df

In [ ]:
def create_real_pred (y1, y2):
    "Prend 2 listes de prédictions et effectue les changements nécessaire pour obtenir les caractères souhaités"
    
    #Conversion des prédictions en ascii
    y1_asci = convert_ascii(y1)
    y2_asci = convert_ascii(y2)
    
    #Opération d'addition des deux images
    somme_ascii = [a + b for a, b in zip(y1_asci, y2_asci)]
    
    #Permet de gérer les valeurs asci > 122
    ascii = []
    for elem in somme_ascii:
        while elem > 122:
            elem -= 65
        ascii.append(elem)
        
    #converti les valeurs ascii par leur caractère correspondant
    real_pred = convert_asci_lettre(ascii)
    return real_pred

def convert_asci_lettre(y_pred):
    "Permet de convertir un vecteur de valeurs ascii [65-122] à une liste de caractères correspondant ascii"
    y_convert = [x - 65 for x in y_pred]
    y_preds = pd.DataFrame({'label' : y_convert})
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz"
    resultat = y_preds['label'].apply(lambda x: alphabet[x])
    return resultat

def convert_ascii(y_pred):
    "Converti une liste de prédictions [0-25] en liste correspondante ASCII"
    liste_ascii = [ord('A') + i  for i in y_pred]
    return liste_ascii


# au cas ou
def convert_lettre(y_pred):
    "Permet de convertir un vecteur de prédiction [0-25] à une liste [A-Z]"
    y_preds = pd.DataFrame({'label' : y_pred})
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"    
    resultat = y_preds['label'].apply(lambda x: alphabet[x])
    return resultat

In [ ]:
pred_final = create_real_pred(grouped_df['label_a'], grouped_df['label_b'])

In [ ]:
submit = pd.concat([grouped_df['id'], pred_final], axis = 1)

In [ ]:
submit.to_csv("submit_perfect.csv", index=False)